In [106]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

In [107]:
#Neural Networks   
    
num_neuron = 10

class Net2(nn.Module):
    # define nn
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(71, num_neuron)      
        self.fc5 = nn.Linear(num_neuron, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        
        X = self.fc1(X)
        X = self.fc5(X)
        X = self.softmax(X)

        return X

In [108]:
# load dataset
PATH = 'depression_new_del600'
dataset = pd.read_csv('dataset/75/'+PATH+'.csv')
dataset.shape

dataset.head()

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,...,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,679,50,11-Dec-61,1,55.0,1,4,6,1,0,...,0,0.0,0,0.0,0.0,1,0.0,0,0,2
1,667,95,26-Sep-61,1,17.0,1,2,3,8,0,...,0,0.0,0,0.0,0.0,0,0.0,0,0,3
2,1160,192,23-Oct-61,1,25.0,1,3,5,9,3,...,0,0.0,0,0.0,0.0,0,0.0,0,0,2
3,419,22,27-Apr-60,1,47.0,1,1,3,12,1,...,0,0.0,0,0.0,0.0,0,0.0,1,0,4
4,217,9,19-May-60,1,28.0,0,2,3,10,2,...,0,0.0,0,0.0,0.0,0,0.0,1,0,5


In [109]:
Xdata = dataset.drop(['surveyid','survey_date', 'village','depressed'],axis = 1)
ydata = dataset[['depressed']]


train_X, test_X, train_y, test_y = train_test_split(Xdata.values,
                                                    dataset.depressed.values)

#torch.from_numpy(train_X)
# wrap up with Variable in pytorch
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())

train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())


In [110]:
net = Net2()

criterion = nn.CrossEntropyLoss()# cross entropy loss

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_app = []

for epoch in range(5000):
    optimizer.zero_grad()
    #train_X, train_y = train_X.to(device), train_y.to(device)
    out = net(train_X)
    #print(out)
    loss = criterion(out, train_y)
    
    #print(loss)
    loss.backward()
    #print(loss.backward())
    
    optimizer.step()
    
    if epoch % 100 == 0:
        print('number of epoch', epoch, 'loss', loss.data)
        loss_app.append(loss.detach().numpy())
        
#test_X,test_y =         
predict_out = net(test_X)
_, predict_y = torch.max(predict_out, 1)

print('prediction accuracy', accuracy_score(test_y.data, predict_y.data))

print('macro precision', precision_score(test_y.data, predict_y.data, average='macro'))
print('micro precision', precision_score(test_y.data, predict_y.data, average='micro'))
print('macro recall', recall_score(test_y.data, predict_y.data, average='macro'))
print('micro recall', recall_score(test_y.data, predict_y.data, average='micro'))

print('precision', precision_score(test_y.data, predict_y.data,average='binary'))

print('recall', recall_score(test_y.data, predict_y.data))


number of epoch 0 loss tensor(0.8200)
number of epoch 100 loss tensor(0.6236)
number of epoch 200 loss tensor(0.6191)
number of epoch 300 loss tensor(0.6160)
number of epoch 400 loss tensor(0.6142)
number of epoch 500 loss tensor(0.6131)
number of epoch 600 loss tensor(0.6122)
number of epoch 700 loss tensor(0.6116)
number of epoch 800 loss tensor(0.6105)
number of epoch 900 loss tensor(0.6100)
number of epoch 1000 loss tensor(0.6094)
number of epoch 1100 loss tensor(0.6087)
number of epoch 1200 loss tensor(0.6085)
number of epoch 1300 loss tensor(0.6079)
number of epoch 1400 loss tensor(0.6081)
number of epoch 1500 loss tensor(0.6073)
number of epoch 1600 loss tensor(0.6075)
number of epoch 1700 loss tensor(0.6070)
number of epoch 1800 loss tensor(0.6066)
number of epoch 1900 loss tensor(0.6066)
number of epoch 2000 loss tensor(0.6063)
number of epoch 2100 loss tensor(0.6066)
number of epoch 2200 loss tensor(0.6061)
number of epoch 2300 loss tensor(0.6058)
number of epoch 2400 loss te

In [111]:
out = net(test_X)

def confusion_matrix(preds, labels):

    preds = torch.argmax(preds, 1)
    conf_matrix = torch.zeros(2, 2)
    for p, t in zip(preds, labels):
        conf_matrix[p, t] += 1

    print(conf_matrix)
    TP = conf_matrix.diag()
    for c in range(1):
        idx = torch.ones(2).byte()
        idx[c] = 0
        TN = conf_matrix[idx.nonzero()[:,None], idx.nonzero()].sum()
        FP = conf_matrix[c, idx].sum()
        FN = conf_matrix[idx, c].sum()

        sensitivity = (TP[c] / (TP[c]+FN))
        specificity = (TN / (TN+FP))

        print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
            c, TP[c], TN, FP, FN))
        print('Sensitivity = {}'.format(sensitivity))
        print('Specificity = {}'.format(specificity))
    return  TP[c], TN, FP, FN



TP,TN,FP,FN = confusion_matrix(out, test_y)
TP,TN,FP,FN = int(TP),int(TN),int(FP),int(FN)

numpy_loss_history = np.array(loss_app)
#print(numpy_loss_history)

np.savetxt("Data = "+PATH +",   No.neuron="+ str(num_neuron)+", acc"+str(acc*100)+", TP="+ str(TP)+", TN="+str(TN)+", FP="+str(FP) + ", FN="+str(FN)+", loss inside"+".txt",numpy_loss_history,delimiter=",")
#save for collect data and loss

tensor([[75., 45.],
        [ 9.,  7.]])


C:\w\b\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\b\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Class 0
TP 75.0, TN 7.0, FP 45.0, FN 9.0
Sensitivity = 0.8928571343421936
Specificity = 0.13461539149284363
